<a href="https://colab.research.google.com/github/AlexSgt/kohya_ss_colab/blob/master/Colab_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kohya_ss GColab version by [AlexS](https://t.me/alaimps)**
youtube.png **[YouTube](https://www.youtube.com/@Alex_Sizov)** | 
boosty.png **[Boosty](https://boosty.to/alexsizov)** | 
telegram.png **[Telegram](https://t.me/aiartistry)** | 
GitHub.png **[GitHub](https://github.com/AlexSgt/kohya_ss_colab)** 

In [ ]:
#@markdown # Install/Update Kohya_ss WebUI
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

#@markdown Save to GDrive
gdrive = False #@param {type: "boolean"}

if gdrive:
  from google.colab import drive
  drive.mount('/content/gdrive')

if not gdrive:
    print('[1;31mGdrive not connected, using colab storage ...')
    time.sleep(4)
    !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
    def inf(msg, style, wdth):
        inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
        display(inf)

    %mkdir -p /content/gdrive/MyDrive/sd
    %cd /content/gdrive/MyDrive/sd
    !git clone https://github.com/AlexSgt/kohya_ss_colab kohya_ss
    !mkdir -p /content/gdrive/MyDrive/sd/kohya_ss/cache/huggingface
    !ln -s /content/gdrive/MyDrive/sd/kohya_ss/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/kohya_ss/
    !git reset --hard
    time.sleep(1)
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/sd/kohya_ss/
    # !pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
    !pip install -r requirements.txt

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Presetting of trigger word and steps
import os
from google.colab import files
import shutil

folders = ['input', 'output', 'log', 'config']
base_path = '/content/gdrive/MyDrive/sd/kohya_ss/Lora/'
trigger_word = "" #@param {type:"string"}

for folder in folders:
  os.makedirs(base_path + folder, exist_ok=True)

input_folder_path = base_path + folders[0] 
print(f"Uploading instance images to {input_folder_path}")
uploaded = files.upload()

num_files = len([k for k in uploaded.keys() if '.txt' not in k])
CONST = 1500

#If your value is less than %step_limit%, then %step_limit% will be the default
step_limit = 100 #@param {type:"integer"}

if int(CONST / num_files) < step_limit:
  num_f = step_limit
else:
  num_f = int(CONST / num_files)

num_input = input_folder_path + f'/{num_f}_{trigger_word}'
os.makedirs(num_input, exist_ok=True)

for filename in uploaded.keys():
    dst_path = os.path.join(num_input, filename)
    shutil.move(filename, dst_path)

In [ ]:
#@markdown # Start Kohya ss WebUI
import sys
import fileinput
from pyngrok import ngrok, conf

#@markdown - Add credentials to your Gradio interface (optional).
User = "" #@param {type:"string"}
Password = "" #@param {type:"string"}

#@markdown - Input your [ngrok](https://ngrok.com/) [token](https://dashboard.ngrok.com/get-started/your-authtoken) if you want to use ngrok server
Ngrok_token = "" #@param {type:"string"}


if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token) , bind_tls=True).public_url
  
  for line in fileinput.input('/content/gdrive/MyDrive/sd/kohya_ss/kohya_gui.py', inplace=True):
    if line.strip().startswith("print('Load CSS...')"):
      line = f"            print('Load CSS...')\n            print('Running on ngrok URL:  {srv}')\n"
    sys.stdout.write(line)
else:
  for line in fileinput.input('/content/gdrive/MyDrive/sd/kohya_ss/kohya_gui.py', inplace=True):
    if line.strip().startswith("print('Running on ngrok URL:"):
      line = f""
    sys.stdout.write(line)

if User and Password:
    !python /content/gdrive/MyDrive/sd/kohya_ss/kohya_gui.py --username $User --password $Password
else:
    !python /content/gdrive/MyDrive/sd/kohya_ss/kohya_gui.py